In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-data/newData.csv
/kaggle/input/checku/top20
/kaggle/input/glove6b50dtxt/glove.6B.50d.txt


In [2]:
# df = pd.read_csv('../input/twitter-data/newData.csv')

In [3]:
# df

In [4]:
# df['label_int'].value_counts()[:20]

In [5]:
# from imblearn.under_sampling import RandomUnderSampler
# from collections import Counter
# top20 = [26, 57, 53, 29, 43, 11, 34, 5, 36, 25, 63, 37, 33, 8, 42, 59, 40,22, 38, 30]
# df = df[df['label_int'].isin(top20)]

In [6]:
# newLabel = df['label_int'].to_numpy()
# text = df['pre_punc'].to_numpy()

In [7]:
# len(text)

In [8]:
# text =np.reshape(text, (-1, 1))

In [9]:
# text.shape

In [10]:
# rus = RandomUnderSampler() 
# resampling X, y
# X_rus, y_rus = rus.fit_resample(text, newLabel)
# new class distribution
# print(Counter(y_rus))

In [11]:
# X_rus = X_rus.flatten()

In [12]:
# X_rus

In [13]:
# list_of_tuples = list(zip(X_rus, y_rus))
# newDf = pd.DataFrame(data = list_of_tuples,columns=['text','emoji_label'])

In [14]:
# newDf

In [15]:
# dicto = {
#     26 :0, 57:1, 53:2, 29:3, 43:4, 11:5, 34:6, 5:7, 36:8, 25:9, 63:10, 37:11, 33:12, 8:13, 42:14, 59:15, 40:16,22:17, 38:18, 30:19
# }

In [16]:
# newDf['emoji_label'] = newDf['emoji_label'].map(dicto)

In [17]:
# newDf

In [18]:
# import nltk
# nltk.download('omw-1.4')
# import spacy
# from nltk.stem import WordNetLemmatizer

# nlp = spacy.load("en_core_web_sm")
# lemmatizer = WordNetLemmatizer()

# def tweet_preprocessing(tweet, nlp=nlp, lemmatizer=lemmatizer):
    
#     stopwords = nltk.corpus.stopwords.words('english')
#     newStopWords = ['url', 'email', 'percent', 'money', 'phone','user','time','date', 'number']
#     stopwords.extend(newStopWords)
#     stop_words = set(stopwords) - set(['not','no','nor'])
#     rm_stop_words_lemma = []
    
#     doc = nlp(tweet)
#     for token in doc:                                            # tokenization
#         if token.text not in stop_words and token.pos_!="PUNCT": 
#             lemma_word = lemmatizer.lemmatize(token.text)        # removing stop
#             rm_stop_words_lemma.append(lemma_word)               # lemmatizing words 
            
#     processed = " ".join(rm_stop_words_lemma)
#     return processed

In [19]:
# newDf["text"] = newDf["text"].apply(tweet_preprocessing)

In [20]:
# def removeNoise(text):
#     temp = text.split()
#     temp_list = []
#     for word in temp:
#         if len(word)>2:
#             temp_list.append(word)
#     return ' '.join(temp_list)

In [21]:
# newDf["text"] = newDf["text"].apply(removeNoise)

In [22]:
# newDf.head()

In [23]:
# newDf.to_csv('top20',index=False)

In [24]:
newDf = pd.read_csv('../input/checku/top20')

In [25]:
newDf = newDf[newDf['emoji_label'].isin([0,1,2])]

In [26]:
newDf = newDf.dropna()
newDf=newDf.reset_index(drop=True)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
c_vect = CountVectorizer()
X = c_vect.fit_transform(newDf["text"])

In [28]:
from sklearn.model_selection import train_test_split
y = newDf["emoji_label"].values
X_train, X_test ,y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=2022,stratify=newDf['emoji_label'])

In [29]:
X_train

<18857x17295 sparse matrix of type '<class 'numpy.int64'>'
	with 133114 stored elements in Compressed Sparse Row format>

In [30]:
print("Training X Dim ",X_train.shape)
print("Training y Dim ",y_train.shape)

Training X Dim  (18857, 17295)
Training y Dim  (18857,)


In [31]:
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()
NB_classifier.fit(X_train, y_train)

MultinomialNB()

In [32]:
from sklearn.metrics import classification_report
y_predict_test = NB_classifier.predict(X_test)
print(classification_report(y_test, y_predict_test))

              precision    recall  f1-score   support

           0       0.52      0.48      0.50      1110
           1       0.45      0.43      0.44      1109
           2       0.44      0.49      0.47      1109

    accuracy                           0.47      3328
   macro avg       0.47      0.47      0.47      3328
weighted avg       0.47      0.47      0.47      3328



In [33]:
# from sklearn.svm import SVC
# clf = SVC()
# clf.fit(X_train, y_train)

In [34]:
# y_predict_test = clf.predict(X_test)
# print(classification_report(y_test, y_predict_test))

In [35]:
from sklearn.model_selection import train_test_split 
train_data, test_data = train_test_split(newDf, test_size = 0.3, stratify = newDf.emoji_label)

In [36]:
x_train = train_data['text'].values
y_train = train_data['emoji_label'].values
x_test = test_data['text'].values
y_test = test_data['emoji_label'].values

In [37]:
x_train.shape, x_test.shape

((15529,), (6656,))

In [38]:
f = open("../input/glove6b50dtxt/glove.6B.50d.txt", encoding='utf8')

In [39]:
embedding_index = {}

for line in f:
    values = line.split()
    word = values[0]
    emb = np.array(values[1:], dtype ='float')
    embedding_index[word] = emb

In [40]:
embedding_index['twitter'].shape

(50,)

In [41]:
def get_embedding_output(X):
    maxLen = 20
    embedding_output = np.zeros((len(X), maxLen, 50))
    
    for ix in range(X.shape[0]):
        my_example = X[ix].split()
        
        for ij in range(len(my_example)): 
            if (embedding_index.get(my_example[ij].lower()) is not None) and (ij<maxLen):
                embedding_output[ix][ij] = embedding_index[my_example[ij].lower()]
            
    return embedding_output

In [42]:
x_train_embed = get_embedding_output(x_train)
x_test_embed = get_embedding_output(x_test)

In [43]:
x_train_embed.shape

(15529, 20, 50)

In [44]:
from tensorflow.keras.utils import to_categorical

In [45]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [46]:
y_train[1]

array([0., 0., 1.], dtype=float32)

In [47]:
# from imblearn.over_sampling import SMOTE

In [48]:
x_train_embed.shape

(15529, 20, 50)

In [49]:
# x_train_embed = x_train_embed.reshape(-1, 1000)
# x_test_embed = x_test_embed.reshape(-1, 1000)

In [50]:
# y_train

In [51]:
# oversample = SMOTE()
# X, y = oversample.fit_resample(x_train_embed, y_train)

In [52]:
# X = X.reshape(-1, 20, 50)
# X.shape, y.shape

In [53]:
# from sklearn.model_selection import train_test_split

In [54]:
# x1, x2, y1, y2 = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
# x1.shape

In [56]:
from keras.models import Sequential 
from keras.layers import LSTM, Dense, Dropout, Bidirectional

In [57]:
model = Sequential()
model.add(Bidirectional(LSTM(units = 512, return_sequences=True), input_shape = (20,50)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units=256)))
model.add(Dropout(0.3))
# model.add(Dense(units=128, activation='relu'))
# model.add(Dense(units=64, activation='relu'))
# model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=3, activation='softmax'))
model.summary()

2022-11-17 13:34:25.023302: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 13:34:25.104056: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 13:34:25.104919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 13:34:25.106054: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 20, 1024)          2306048   
_________________________________________________________________
dropout (Dropout)            (None, 20, 1024)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               2623488   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 1539      
Total params: 4,931,075
Trainable params: 4,931,075
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['accuracy'])

In [59]:
hist = model.fit(x_train_embed, y_train, validation_split=0.2, shuffle=True, batch_size=64, epochs=25)

2022-11-17 13:34:28.805130: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-11-17 13:34:34.188956: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


195/195 [==============================] - 13s 31ms/step - loss: 1.0830 - accuracy: 0.4008 - val_loss: 1.0877 - val_accuracy: 0.4034
Epoch 2/25
195/195 [==============================] - 4s 21ms/step - loss: 1.0662 - accuracy: 0.4336 - val_loss: 1.0715 - val_accuracy: 0.4082
Epoch 3/25
195/195 [==============================] - 4s 20ms/step - loss: 1.0557 - accuracy: 0.4453 - val_loss: 1.0780 - val_accuracy: 0.4102
Epoch 4/25
195/195 [==============================] - 4s 21ms/step - loss: 1.0450 - accuracy: 0.4542 - val_loss: 1.0661 - val_accuracy: 0.4276
Epoch 5/25
195/195 [==============================] - 4s 20ms/step - loss: 1.0332 - accuracy: 0.4688 - val_loss: 1.0692 - val_accuracy: 0.4263
Epoch 6/25
195/195 [==============================] - 4s 20ms/step - loss: 1.0146 - accuracy: 0.4842 - val_loss: 1.0741 - val_accuracy: 0.4330
Epoch 7/25
195/195 [==============================] - 4s 21ms/step - loss: 0.9945 - accuracy: 0.5041 - val_loss: 1.0752 - val_accuracy: 0.4263
Epoch 8/2

KeyboardInterrupt: 

In [60]:
model.evaluate(x_test_embed, y_test)

208/208 [==============================] - 2s 6ms/step - loss: 1.4901 - accuracy: 0.4109


[1.4900681972503662, 0.41090744733810425]

In [61]:
y_pred = model.predict(x_test_embed)
from sklearn.metrics import classification_report
y_pred = np.array([np.argmax(pred) for pred in y_pred])
y_test = np.array([np.argmax(test) for test in y_test])
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.42      0.50      0.46      2219
           1       0.39      0.30      0.34      2219
           2       0.41      0.43      0.42      2218

    accuracy                           0.41      6656
   macro avg       0.41      0.41      0.41      6656
weighted avg       0.41      0.41      0.41      6656



In [ ]:
# model.save('BiLSTM.h5')